In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import glob
from collections import Counter
import math
import re
import json
import subprocess
import time
import pickle

import pandas as pd
import umap
from tqdm.autonotebook import tqdm, trange
from nltk.tokenize import word_tokenize

import numpy as np
import scipy.stats
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split, KFold

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score
from sklearn.utils import class_weight

from tensorflow import keras

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
#assert gpus
try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
layers = keras.layers
models = keras.models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import numpy.random as rng
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint

import convert_model

import language
import text_nn
import grab_category
import news
import groups
import libs.cpp_stuff as cpp

In [3]:
fi = language.read_dump("data/website/en/categories_en/society")
cpp.load_idf(groups.counts_for_grouping_en)

In [4]:
title_fi = {}
for f in fi:
    if f.title not in title_fi:
        title_fi[f.title] = []
    
    title_fi[f.title].append(f)

In [5]:
articles_1 = """	5 dead, many more missing in eruption of New Zealand volcano site:ABC Action News - WFTS - Tampa Bay
	Five dead as volcano erupts in New Zealand site:Lancashire Post
	At least five dead, many more missing following eruption of New Zealand volcano site:The Japan Times
	5 dead, many more missing in eruption of New Zealand volcano site:Bashaw Star
	5 dead, many more missing in eruption of New Zealand volcano site:Barriere Star Journal
	5 dead, many more missing in eruption of New Zealand volcano site:The Stettler Independent
	Volcano Erupts in New Zealand; 5 Dead, Many Missing site:Courthouse News Service
	iN VIDEO: 5 dead, many more missing in eruption of New Zealand volcano site:iNFOnews.ca
	5 dead, many more missing or injured after New Zealand volcano erupts site:Japan Today
	Terrifying footage of New Zealand volcano eruption that has killed five site:PlymouthLive
	Five dead as volcano erupts in New Zealand site:Shropshire Star
	1 dead, many more missing in eruption of New Zealand volcano site:An-Nahar
	Five dead as volcano erupts in New Zealand site:Express & Star
	5 dead, many more missing in eruption of New Zealand volcano site:North Delta Reporter
	Dozens feared dead in eruption of New Zealand volcano site:THE OKLAHOMAN
	5 dead, many missing in eruption of NZ volcano; search ops yet to begin site:Business-Standard
	5 dead, 8 missing in New Zealand volcano eruption site:Las Vegas Review-Journal
	New Zealand volcano eruption leaves at least five dead after tourists seen in crater caught up in explosion site:DevonLive
	Britons feared missing after volcano erupts in New Zealand site:Shropshire Star
	Deadly volcano eruption site:Castanet
	UPDATE: Five dead, many more missing in eruption of New Zealand volcano | IOL News site:www.iol.co.za
	Britons feared missing after volcano erupts in New Zealand site:Express & Star
	Tourists missing after deadly New Zealand volcano eruption site:Times of Oman
	New Zealand: Five dead after volcanic island erupts - British tourist among missing site:THE SCOTSMAN"""

articles_1 = [a.strip().split(" site:") for a in articles_1.split("\n")]
fi_1 = []
for title, site in articles_1:
    for f in title_fi[title]:
        if f.site == site:
            fi_1.append(f)
            break
            
len(fi_1)

24

In [6]:
articles_2 = """	Five dead, many injured after New Zealand volcano eruption site:The Guardian
	One critically injured as popular New Zealand volcano erupts site:The Peninsula
	UPDATE: Five dead, many injured after New Zealand volcano eruption site:News24
	Five dead, many injured after New Zealand volcano eruption site:Punch Newspapers
	Five dead as popular New Zealand volcano erupts site:The Peninsula
	New Zealand volcano eruption kills 5 site:Daily Nation
	Several injured, about 100 people in the area as New Zealand's White Island volcano erupts site:CNA
	Five dead, tourists stranded as New Zealand volcano erupts site:Saudi Gazette
	1 dead as NZ volcano erupts, more fatalities likely site:Deccan Herald
	Injuries feared as popular New Zealand volcano erupts site:Deccan Herald
	No more survivors from New Zealand volcano eruption site:Daily Nation"""

articles_2 = [a.strip().split(" site:") for a in articles_2.split("\n")]
fi_2 = []
for title, site in articles_2:
    for f in title_fi[title]:
        if f.site == site:
            fi_2.append(f)
            break
            
len(fi_2)

11

In [76]:
fis = fi_1 + fi_2
gg = cpp.group_texts(groups.extract_infos_for_grouping(fis), groups.counts_for_grouping_en, 0.2)

for g in gg:
    for i in g:
        print(f"#{i}", fis[i].title, "site:", fis[i].site)
    
    print()
    print()

#19 Deadly volcano eruption site: Castanet
#16 5 dead, 8 missing in New Zealand volcano eruption site: Las Vegas Review-Journal
#3 5 dead, many more missing in eruption of New Zealand volcano site: Bashaw Star
#13 5 dead, many more missing in eruption of New Zealand volcano site: North Delta Reporter
#5 5 dead, many more missing in eruption of New Zealand volcano site: The Stettler Independent
#4 5 dead, many more missing in eruption of New Zealand volcano site: Barriere Star Journal
#15 5 dead, many missing in eruption of NZ volcano; search ops yet to begin site: Business-Standard
#14 Dozens feared dead in eruption of New Zealand volcano site: THE OKLAHOMAN
#12 Five dead as volcano erupts in New Zealand site: Express & Star
#10 Five dead as volcano erupts in New Zealand site: Shropshire Star
#1 Five dead as volcano erupts in New Zealand site: Lancashire Post
#9 Terrifying footage of New Zealand volcano eruption that has killed five site: PlymouthLive
#0 5 dead, many more missing in er

In [27]:
for (i1, i2), sim in sorted(cpp.calc_similarities(groups.extract_infos_for_grouping(fis), groups.counts_for_grouping_en, 0.2), key=lambda x: x[1], reverse=False)[:10]:
    print(f"#{i1}", fis[i1].title, "site:", fis[i1].site)    
    print(f"#{i2}", fis[i2].title, "site:", fis[i2].site)
    print(sim)
    print()
    print()

#29 New Zealand volcano eruption kills 5 site: Daily Nation
#34 No more survivors from New Zealand volcano eruption site: Daily Nation
0.0


#32 1 dead as NZ volcano erupts, more fatalities likely site: Deccan Herald
#33 Injuries feared as popular New Zealand volcano erupts site: Deccan Herald
0.0


#10 Five dead as volcano erupts in New Zealand site: Shropshire Star
#18 Britons feared missing after volcano erupts in New Zealand site: Shropshire Star
0.0


#12 Five dead as volcano erupts in New Zealand site: Express & Star
#21 Britons feared missing after volcano erupts in New Zealand site: Express & Star
0.0


#20 UPDATE: Five dead, many more missing in eruption of New Zealand volcano | IOL News site: www.iol.co.za
#30 Several injured, about 100 people in the area as New Zealand's White Island volcano erupts site: CNA
0.061811916530132294


#20 UPDATE: Five dead, many more missing in eruption of New Zealand volcano | IOL News site: www.iol.co.za
#25 One critically injured as popular N

In [21]:
chunks_1 = [cpp.get_chunks(words, groups.counts_for_grouping_en) for words in groups.extract_texts_for_grouping(fi_1)]
chunks_2 = [cpp.get_chunks(words, groups.counts_for_grouping_en) for words in groups.extract_texts_for_grouping(fi_2)]
chunkss = chunks_1 + chunks_2

In [176]:
min_idf = 1
def chunks_total(chunks):
    return sum(count * idf for _, (count, idf) in chunks.items() if idf > min_idf)

total_1 = [chunks_total(chunks) for chunks in chunks_1]
total_2 = [chunks_total(chunks) for chunks in chunks_2]
totals = total_1 + total_2

In [177]:
def calc_sim(chunks1, chunks2, t1, t2):
    s = 0
    for chunk, (count, idf) in chunks1.items():
        if chunk in chunks2:
            if idf > min_idf:
                s += idf * min(count, chunks2[chunk][0])
            
    return s / max(t1, t2)

def simn(i1, i2):
    return calc_sim(chunkss[i1], chunkss[i2], totals[i1], totals[i2])

In [178]:
simn(20, 32)

0.09106980070838247

In [179]:
def join_chunks(chunkss, totals):
    res = {}
    sum_total = sum(totals)
    for chunks, total in zip(chunkss, totals):
        weight = 1 / len(totals)
        for chunk, (count, idf) in chunks.items():
            if chunk not in res:
                res[chunk] = 0, idf
                
            res[chunk] = res[chunk][0] + count * weight, idf
    
    return res, chunks_total(res)# sum_total / len(totals)

chunksj1, totalj1 = join_chunks(chunks_1, total_1)
chunksj2, totalj2 = join_chunks(chunks_2, total_2)

In [180]:
for i in range(24, 24 + 11):
    print(f"#{i}", calc_sim(chunksj1, chunkss[i], totalj1, totals[i]))
    
print("1 - 2:", calc_sim(chunksj1, chunksj2, totalj1, totalj2))

#24 0.18358272898321953
#25 0.15139506016183346
#26 0.18766340554311078
#27 0.18695828554595278
#28 0.1560199991995665
#29 0.1751668894208689
#30 0.15418748996532244
#31 0.19005115881378515
#32 0.17333083325985008
#33 0.14277868551136236
#34 0.18680661928931033
1 - 2: 0.23786758586326023


In [181]:
print(chunks_total(chunksj1), totalj1)

4684.061293780824 4684.061293780824


In [182]:
chunks_by_gg = []
totals_gg = []
for g in gg:
    c, t = join_chunks([chunkss[i] for i in g], [totals[i] for i in g])
    chunks_by_gg.append(c)
    totals_gg.append(t)

In [183]:
for i in range(len(gg)):
    for j in range(i + 1, len(gg)):
        print(f"{i} - {j}: {calc_sim(chunks_by_gg[i], chunks_by_gg[j], totals_gg[i], totals_gg[j])}")

0 - 1: 0.23254316202633915
0 - 2: 0.2609407910579921
1 - 2: 0.17357404311111538


In [184]:
def make_groups(chunkss, totals, indices):
    groups = []
    for i, c, t in zip(indices, chunkss, totals):
        joined = False
        for j, (cg, tg, idx) in enumerate(groups):
            if calc_sim(c, cg, t, tg) >= 0.2:
                groups[j] = *join_chunks([c, cg], [t, tg]), idx + i
                joined = True
                break

        if not joined:
            groups.append((c, t, i))
    
    return groups

In [185]:
gg1 = make_groups(chunkss, totals, [[i] for i in range(len(totals))])
for _, _, g in gg1:
    print(g)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23]
[20]
[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]


In [186]:
gg2 = make_groups(*zip(*gg1))
for _, _, g in gg2:
    print(g)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
[20]


In [133]:
gg3 = make_groups(*zip(*gg2))
for _, _, g in gg3:
    print(g)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 19, 17, 22, 24, 26, 27, 29, 31, 32, 33, 34]
[18, 21, 23]
[20]
[25, 28, 30]


In [121]:
gg4 = make_groups(*zip(*gg3))
for _, _, g in gg4:
    print(g)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 22, 32, 33, 24, 26, 27, 29, 31, 34, 25, 28, 30]
[18, 21, 23]
[20]
